# Pinguinos

## Librerias y Dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import cross_validate


from sklearn.model_selection import GridSearchCV

#se importa el archivo cvc
df=pd.read_csv("/content/drive/MyDrive/IA/Pinguins/penguins_size.csv", sep=',')
#df.head()
#print(df.columns)


## Limpia de datos

Se separan los datos de entrada-salida a clasificar

In [3]:
DataTrain=df[['island', 'culmen_length_mm', 'culmen_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex']]
#print(DataTrain.head())

Out_T=df['species']
#DataTrain.head()

Se combierten los valores escritos a estados numericos 

In [4]:
%%capture
#Remplazamos en la columna 'sex' los valores: 'male' y 'female' por 1 y -1 respectivamente
DataTrain['sex']=DataTrain['sex'].replace(to_replace=['MALE','FEMALE'], value=[1,-1])


#Remplazamos en la columna 'island' los valores: 'Biscoe', 'Dream' y 'Torgersen' por 1 2 y 3 respectivamente
DataTrain['island']=DataTrain['island'].replace(to_replace=['Biscoe','Dream','Torgersen'], value=[1,2,3])


Se verifican los NaN 

In [5]:

#buscamos NaN values
for colum in DataTrain.columns:
  print(str(colum)+': '+str(DataTrain[colum].isnull().sum()) )



island: 0
culmen_length_mm: 2
culmen_depth_mm: 2
flipper_length_mm: 2
body_mass_g: 2
sex: 11


Se verifican y eliminan las filas que estan muy vacias

In [6]:
RowsToDelete=[]
for i in range(len(DataTrain)): 
  CantNull=(DataTrain.loc[i,['culmen_length_mm', 'culmen_depth_mm','flipper_length_mm', 'body_mass_g', 'sex']].isnull().sum())
  if CantNull>2:
    RowsToDelete.append(i)
    print("The index "+str(i)+" have a " +str(CantNull)+ " NaN values")

#elimina filas en data train
DataTrain = DataTrain.drop(RowsToDelete)
DataTrain = DataTrain.reset_index()
DataTrain = DataTrain.drop("index", axis=1)
#elimina filas en data out
Out_T = Out_T.drop(RowsToDelete)
Out_T = Out_T.reset_index()
Out_T= Out_T.drop("index", axis=1)


#DataTrain.head(15)

The index 3 have a 5 NaN values
The index 339 have a 5 NaN values


Se rellenan los NaN con el promedio de la isla

In [7]:
#reemplazamos los NaN con la media de la isla

let1=DataTrain['sex'].where(DataTrain['island'] ==1).mean()
let2=DataTrain['sex'].where(DataTrain['island'] ==2).mean()
let3=DataTrain['sex'].where(DataTrain['island'] ==3).mean()

print(let1,let2,let3)
DataTrain['sex'].mask( (DataTrain['island'] ==1) & ( pd.isna(DataTrain['sex'])), let1, inplace=True )
DataTrain['sex'].mask( (DataTrain['island'] ==2) & ( pd.isna(DataTrain['sex'])), let2, inplace=True )
DataTrain['sex'].mask( (DataTrain['island'] ==3) & ( pd.isna(DataTrain['sex'])), let3, inplace=True )
#DataTrain.head(15)


0.018404907975460124 0.008130081300813009 -0.02127659574468085


## One_HOT

In [8]:
ohe = OneHotEncoder()
X=np.array([*Out_T.values])
print(X.shape)
ohe.fit(X)
transformed=ohe.transform(X)
Out_MClass=transformed.toarray()
print(Out_MClass.shape)
#print(Out_MClass)

(342, 1)
(342, 3)


## Division de Casos (10,90) - (Validation,Train)

In [9]:

Data_Train,Data_Validation,Out_Train,Out_Validation = train_test_split(DataTrain,Out_MClass,test_size=0.1)
#print(Out_Train)
#print(DataTrain.shape)
#print(Datanorm_Validation.shape)

## Normalizacion

Se hace la normalizacion

In [10]:
Data=np.array(Data_Train.values)
scaler = StandardScaler()
scaler.fit(Data)
MeanTrain=scaler.mean_
VarTrain=scaler.var_
Datanorm_Train=scaler.transform(Data)
print(Datanorm_Train.shape)

(307, 6)


## Validacion Cruzada

In [11]:

verbose=False
clf = MLPClassifier(hidden_layer_sizes=(8, 8),activation='relu',random_state=1,learning_rate_init=0.005, max_iter=300,verbose=verbose);

##clf.fit(X_train, y_train)
##Datanorm_Train, Out_Train=shuffle( Datanorm_Train, Out_Train, random_state=12)
#print(Out_Train)
cv_results = cross_validate(clf, Datanorm_Train, Out_Train, cv=4);
# sorted(cv_results.keys())
print("test scores:",*cv_results['test_score'])
#['identity', 'logistic', 'relu', 'softmax', 'tanh'].

test scores: 1.0 1.0 1.0 0.9868421052631579


De acuerdo a la validacion cruzada, podemos decir que la red neuronal funciona correctamente para todo el conjunto de datos

## Busqueda exhaustiva

  Se hace el grid search

In [12]:
%%capture
clf = MLPClassifier(random_state=1 ,verbose=verbose)
parameters = {'activation':('identity', 'logistic', 'relu', 'softmax', 'tanh'),'hidden_layer_sizes': [(4,4),(6,6),(8,8)], 'learning_rate_init':[0.002,0.003,0.004,0.005],'max_iter':[200,350,500]}

clf_grid = GridSearchCV(clf, parameters)
clf_grid.fit(Datanorm_Train, Out_Train);



Se verifican los resultados y se almacenan en un dataframe

In [13]:
#print(sorted(clf_grid.cv_results_.keys()))

data=clf_grid.cv_results_

dfit = pd.DataFrame(data)


Se extraen las filas con un score rank igual a 1

In [14]:
dfit.loc[dfit['rank_test_score']==1,['param_activation','param_hidden_layer_sizes', 'param_learning_rate_init','param_max_iter','mean_fit_time', 'mean_score_time', 'mean_test_score','std_test_score','std_score_time','params']]
#'param_activation', 'param_hidden_layer_sizes', 'param_learning_rate_init','mean_fit_time', 'mean_score_time', 'mean_test_score','std_score_time','params'])

,param_activation,param_hidden_layer_sizes,param_learning_rate_init,param_max_iter,mean_fit_time,mean_score_time,mean_test_score,std_test_score,std_score_time,params
59,logistic,"(6, 6)",0.005,500,0.503963,0.001189,0.996721,0.006557,0.000096,"{'activation': 'logistic', 'hidden_layer_sizes..."
68,logistic,"(8, 8)",0.004,500,0.516782,0.001349,0.996721,0.006557,0.000264,"{'activation': 'logistic', 'hidden_layer_sizes..."
70,logistic,"(8, 8)",0.005,350,0.359781,0.001352,0.996721,0.006557,0.000316,"{'activation': 'logistic', 'hidden_layer_sizes..."
71,logistic,"(8, 8)",0.005,500,0.529213,0.001192,0.996721,0.006557,0.000034,"{'activation': 'logistic', 'hidden_layer_sizes..."
77,relu,"(4, 4)",0.003,500,0.492295,0.001164,0.996721,0.006557,0.000039,"{'activation': 'relu', 'hidden_layer_sizes': (..."
79,relu,"(4, 4)",0.004,350,0.339691,0.001187,0.996721,0.006557,0.000076,"{'activation': 'relu', 'hidden_layer_sizes': (..."
80,relu,"(4, 4)",0.004,500,0.445884,0.001142,0.996721,0.006557,0.000009,"{'activation': 'relu', 'hidden_layer_sizes': (..."
82,relu,"(4, 4)",0.005,350,0.350710,0.001254,0.996721,0.006557,0.000094,"{'activation': 'relu', 'hidden_layer_sizes': (..."
83,relu,"(4, 4)",0.005,500,0.376146,0.001158,0.996721,0.006557,0.000049,"{'activation': 'relu', 'hidden_layer_sizes': (..."
100,relu,"(8, 8)",0.003,350,0.384602,0.001181,0.996721,0.006557,0.000054,"{'activation': 'relu', 'hidden_layer_sizes': (..."


En primer lugar se hace el ranking de acuerdo al score, donde 30 conjuntos de hiperparametros estan en el 1'st rank.

-Se escoge el conjunto de hiperparanetros en el index 79 pues solo usa 2 capas de a 4 neuronas, una cantidad de iteraciones igual a 350 y el ratio de crecimiento igual a 0.004. El resto de las estadisticas son muy similares como para dar

## Re-Entrenamiento

In [15]:
%%capture
Parametros=dfit.loc[79,'params']
print(Parametros)
clf = MLPClassifier(**Parametros,random_state=1 ,verbose=verbose)
clf.fit(Datanorm_Train, Out_Train)


## Validacion

In [16]:
DataT=np.array(Data_Validation)
Datanorm_Validation=scaler.transform(DataT)

scikit_score =clf.score(Datanorm_Validation,Out_Validation)
print("Score:",scikit_score)

Score: 1.0


## Conclusiones

-Fue muy importante realizar una validacion cruzada, pues en este caso hay muy poquitos datos

-Usar el metodo de gridSearch es extremadamente util pues permite encontrar el conjunto de hiperparametros mas adecuado para el Modelo

-En este caso dio Un Score de 1 para el conjunto de validacion(10%) en donde se evaluo el desempeñ final del modelo, Lo que podria ser debido a que la especie esta estrechamente relacionada con la isla de origen y que el resto de datos son poseen los parametros que varian de especie a especie.